summary: A tutorial to understand the process of building a Neural Matrix Factorization model from scratch in PyTorch on MovieLens-1M dataset.
id: neural-matrix-factorization-from-scratch-in-pytorch
categories: Pytorch
tags: scratch, movielens
status: Published 
authors: Sparsh A.
Feedback Link: https://form.jotform.com/211377288388469

# Neural Matrix Factorization from scratch in PyTorch

In [2]:
# - create a folder Project in drive, copy both data directory and NMF.ipynb notebook to Project folder
# - Then the below code will load the data and use it to run
is_colab = False
if is_colab:
    from google.colab import drive
    drivePrefix = '/content/drive'
    drive.mount(drivePrefix, force_remount=True)

    !cp -r /content/drive/MyDrive/Project/data .

Mounted at /content/drive


<!-- ------------------------ -->
## What you'll learn
Duration: 2

- Create movielens dataset class in Pytorch
- Setting the evaluation criteria
- Architecture of neural matrix factorization model
- Train and evaluating a neural matrix factorization model

<!-- ------------------------ -->
## Dataset
Duration: 5

After downloading and expanding the movielens-1m dataset, we will create the dataset class as the first step.

In [3]:
import torch

In [4]:
class Rating_Datset(torch.utils.data.Dataset):
    def __init__(self, user_list, item_list, rating_list):
        super(Rating_Datset, self).__init__()
        self.user_list = user_list
        self.item_list = item_list
        self.rating_list = rating_list

    def __len__(self):
        return len(self.user_list)

    def __getitem__(self, idx):
        user = self.user_list[idx]
        item = self.item_list[idx]
        rating = self.rating_list[idx]

        return (
            torch.tensor(user, dtype=torch.long),
            torch.tensor(item, dtype=torch.long),
            torch.tensor(rating, dtype=torch.float)
            )

The name of our class is *Rating_Dataset* and it is getting inherited from PyTorch *Dataset* base class. The *__getitem__* method is helping us in 2 ways: 1) It is reinforcing the type to [long, long, float] and returning the tensor version of the tuple for the given index id.

We are also creating a helper dataset class to put all the data processing functions under a single umbrella. This class contains 5 methods:

- *_reindex*: process dataset to reindex userID and itemID, also set rating as binary feedback
- *_leave_one_out*: leave-one-out evaluation protocol in paper [https://www.comp.nus.edu.sg/~xiangnan/papers/ncf.pdf](https://www.comp.nus.edu.sg/~xiangnan/papers/ncf.pdf)
- *_negative_sampling*: randomly selects n negative examples for each positive one
- *get_train_instance*: merge the examples of train data with negative samples and return the PyTorch dataloader object
- *get_test_instance*: merge the examples of test data with negative samples and return the PyTorch dataloader object

<!-- ------------------------ -->
## Evaluation criteria
Duration: 5

Next, we are defining evaluation metrics. We are using Hit Rate and NDCG as our evaluation metrics.

In [5]:
def hit(ng_item, pred_items):
    if ng_item in pred_items:
        return 1
    return 0


def ndcg(ng_item, pred_items):
    if ng_item in pred_items:
        index = pred_items.index(ng_item)
        return np.reciprocal(np.log2(index+2))
    return 0


def metrics(model, test_loader, top_k, device):
    HR, NDCG = [], []

    for user, item, label in test_loader:
        user = user.to(device)
        item = item.to(device)

        predictions = model(user, item)
        _, indices = torch.topk(predictions, top_k)
        recommends = torch.take(
                item, indices).cpu().numpy().tolist()

        ng_item = item[0].item() # leave one-out evaluation has only one item per user
        HR.append(hit(ng_item, recommends))
        NDCG.append(ndcg(ng_item, recommends))

    return np.mean(HR), np.mean(NDCG)

The metrics function is first loading the user and item variables to the right device (e.g. to the GPU if it is enabled), then getting predictions from the model and then finally calculating (& returning) the hit_rate_at_k and ndcg_at_k values.

<!-- ------------------------ -->
## Defining Model Architectures
Duration: 10

After defining the dataset class and evaluation function, it is time to define the model architecture.

We are going to use *Neural Collaborative Filtering for Personalized Ranking*. This model leverages the flexibility and non-linearity of neural networks to replace dot products of matrix factorization, aiming at enhancing the model expressiveness. In specific, this model is structured with two subnetworks including generalized matrix factorization (GMF) and MLP and models the interactions from two pathways instead of simple inner products. The outputs of these two networks are concatenated for the final prediction scores calculation.

![nmf_architecture](img/nmf_architecture.png)

In this architecture, we are first creating the user and item embedding layers for both MLP and MF architectures, and with the help of PyTorch ModuleList, we are creating MLP architecture. Then, in the forward method, we are passing user and item indices list in the embedding layers and then concatenating and multiplying the MLP and MF embedding layers respectively. And finally, concatenating the MLP and MF feature layers and a logistic activation at the end.

In [6]:
import torch.nn as nn

In [7]:
class NeuMF(nn.Module):
    def __init__(self, args, num_users, num_items):
        super(NeuMF, self).__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.factor_num_mf = args.factor_num
        self.factor_num_mlp =  int(args.layers[0]/2)
        self.layers = args.layers
        self.dropout = args.dropout

        self.embedding_user_mlp = nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.factor_num_mlp)
        self.embedding_item_mlp = nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.factor_num_mlp)

        self.embedding_user_mf = nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.factor_num_mf)
        self.embedding_item_mf = nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.factor_num_mf)

        self.fc_layers = nn.ModuleList()
        for idx, (in_size, out_size) in enumerate(zip(args.layers[:-1], args.layers[1:])):
            self.fc_layers.append(torch.nn.Linear(in_size, out_size))
            self.fc_layers.append(nn.ReLU())

        self.affine_output = nn.Linear(in_features=args.layers[-1] + self.factor_num_mf, out_features=1)
        self.logistic = nn.Sigmoid()
        self.init_weight()

    def init_weight(self):
        nn.init.normal_(self.embedding_user_mlp.weight, std=0.01)
        nn.init.normal_(self.embedding_item_mlp.weight, std=0.01)
        nn.init.normal_(self.embedding_user_mf.weight, std=0.01)
        nn.init.normal_(self.embedding_item_mf.weight, std=0.01)

        for m in self.fc_layers:
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)

        nn.init.xavier_uniform_(self.affine_output.weight)

        for m in self.modules():
            if isinstance(m, nn.Linear) and m.bias is not None:
                m.bias.data.zero_()

    def forward(self, user_indices, item_indices):
        user_embedding_mlp = self.embedding_user_mlp(user_indices)
        item_embedding_mlp = self.embedding_item_mlp(item_indices)

        user_embedding_mf = self.embedding_user_mf(user_indices)
        item_embedding_mf = self.embedding_item_mf(item_indices)

        mlp_vector = torch.cat([user_embedding_mlp, item_embedding_mlp], dim=-1)
        mf_vector =torch.mul(user_embedding_mf, item_embedding_mf)

        for idx, _ in enumerate(range(len(self.fc_layers))):
            mlp_vector = self.fc_layers[idx](mlp_vector)

        vector = torch.cat([mlp_vector, mf_vector], dim=-1)
        logits = self.affine_output(vector)
        rating = self.logistic(logits)
        return rating.squeeze()

<!-- ------------------------ -->
## Training and evaluation
Duration: 10

We are using following hyperparameters to train the model:
- Learning rate is 0.001
- Dropout rate is 0.2
- Running for 10 epochs
- HitRate@10 and NDCG@10
- 4 negative samples for each positive one

In [8]:
import argparse
import random
import numpy as np
import pandas as pd
import os
import time

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
MODEL_PATH = ""

In [9]:
parser = argparse.ArgumentParser()
parser.add_argument("--seed", 
    type=int, 
    default=42, 
    help="Seed")
parser.add_argument("--lr", 
    type=float, 
    default=0.001, 
    help="learning rate")
parser.add_argument("--dropout", 
    type=float,
    default=0.2,  
    help="dropout rate")
parser.add_argument("--batch_size", 
    type=int, 
    default=256, 
    help="batch size for training")
parser.add_argument("--epochs", 
    type=int,
    default=30,  
    help="training epoches")
parser.add_argument("--top_k", 
    type=int, 
    default=10, 
    help="compute metrics@top_k")
parser.add_argument("--factor_num", 
    type=int,
    default=32, 
    help="predictive factors numbers in the model")
parser.add_argument("--layers",
    nargs='+', 
    default=[64,32,16,8],
    help="MLP layers. Note that the first layer is the concatenation of user and item embeddings. So layers[0]/2 is the embedding size.")
parser.add_argument("--num_ng", 
    type=int,
    default=4, 
    help="Number of negative samples for training set")
parser.add_argument("--num_ng_test", 
    type=int,
    default=100, 
    help="Number of negative samples for test set")
parser.add_argument("--out", 
    default=True,
    #default=False,
    help="save model or not")

args = parser.parse_args("")

In [10]:
class NCF_Data(object):
    """
    Construct Dataset for NCF
    """
    def __init__(self, args, ratings):
        self.ratings = ratings
        self.num_ng = args.num_ng
        self.num_ng_test = args.num_ng_test
        self.batch_size = args.batch_size

        self.preprocess_ratings = self._reindex(self.ratings)

        self.user_pool = set(self.ratings['user_id'].unique())
        self.item_pool = set(self.ratings['item_id'].unique())

        self.train_ratings, self.test_ratings = self._leave_one_out(self.preprocess_ratings)
        self.negatives = self._negative_sampling(self.preprocess_ratings)
        random.seed(args.seed)

    def _reindex(self, ratings):
        """
        Process dataset to reindex userID and itemID, also set rating as binary feedback
        """
        user_list = list(ratings['user_id'].drop_duplicates())
        user2id = {w: i for i, w in enumerate(user_list)}

        item_list = list(ratings['item_id'].drop_duplicates())
        item2id = {w: i for i, w in enumerate(item_list)}

        ratings['user_id'] = ratings['user_id'].apply(lambda x: user2id[x])
        ratings['item_id'] = ratings['item_id'].apply(lambda x: item2id[x])
        ratings['rating'] = ratings['rating'].apply(lambda x: float(x > 0))
        return ratings

    def _leave_one_out(self, ratings):
        """
        leave-one-out evaluation protocol in paper https://www.comp.nus.edu.sg/~xiangnan/papers/ncf.pdf
        """
        ratings['rank_latest'] = ratings.groupby(['user_id'])['timestamp'].rank(method='first', ascending=False)
        test = ratings.loc[ratings['rank_latest'] == 1]
        train = ratings.loc[ratings['rank_latest'] > 1]
        assert train['user_id'].nunique()==test['user_id'].nunique(), 'Not Match Train User with Test User'
        return train[['user_id', 'item_id', 'rating']], test[['user_id', 'item_id', 'rating']]



    def _negative_sampling(self, ratings):
        interact_status = (
            ratings.groupby('user_id')['item_id']
            .apply(set)
            .reset_index()
            .rename(columns={'item_id': 'interacted_items'}))
        interact_status['negative_items'] = interact_status['interacted_items'].apply(lambda x: self.item_pool - x)
        interact_status['negative_samples'] = interact_status['negative_items'].apply(lambda x: random.sample(x, self.num_ng_test))
        return interact_status[['user_id', 'negative_items', 'negative_samples']]

    def get_train_instance(self):
        users, items, ratings = [], [], []
        train_ratings = pd.merge(self.train_ratings, self.negatives[['user_id', 'negative_items']], on='user_id')
        train_ratings['negatives'] = train_ratings['negative_items'].apply(lambda x: random.sample(x, self.num_ng))
        for row in train_ratings.itertuples():
            users.append(int(row.user_id))
            items.append(int(row.item_id))
            ratings.append(float(row.rating))
            for i in range(self.num_ng):
                users.append(int(row.user_id))
                items.append(int(row.negatives[i]))
                ratings.append(float(0))  # negative samples get 0 rating
        dataset = Rating_Datset(
            user_list=users,
            item_list=items,
            rating_list=ratings)
        return torch.utils.data.DataLoader(dataset, batch_size=self.batch_size, shuffle=True, num_workers=4)

    def get_test_instance(self):
        users, items, ratings = [], [], []
        test_ratings = pd.merge(self.test_ratings, self.negatives[['user_id', 'negative_samples']], on='user_id')
        for row in test_ratings.itertuples():
            users.append(int(row.user_id))
            items.append(int(row.item_id))
            ratings.append(float(row.rating))
            for i in getattr(row, 'negative_samples'):
                users.append(int(row.user_id))
                items.append(int(i))
                ratings.append(float(0))
        dataset = Rating_Datset(
            user_list=users,
            item_list=items,
            rating_list=ratings)
        return torch.utils.data.DataLoader(dataset, batch_size=self.num_ng_test+1, shuffle=False, num_workers=4)

In [11]:
#args = parser.parse_args("")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#writer = SummaryWriter()

# seed for Reproducibility
seed_everything(seed=42)

# load data
if is_colab:
    data_cols = ['user_id', 'item_id', 'rating', 'timestamp']
    ml_100k_training = pd.read_csv('data/MovieLens.training', sep='\t', lineterminator='\n')
    ml_100k_training.columns = data_cols

    ml_100k_test = pd.read_csv('data/MovieLens.test', sep='\t', lineterminator='\n')
    ml_100k_test.columns = data_cols
else:
    ml_100k_training = pd.read_csv("../data/MovieLens.training", sep="\t", names = ['user_id', 'item_id', 'rating', 'timestamp'])
    ml_100k_test = pd.read_csv("../data/MovieLens.test", sep="\t", names = ['user_id', 'item_id', 'rating', 'timestamp'])

ml_100k = ml_100k_training
ml_100k

# set the num_users, items
num_users = ml_100k['user_id'].nunique()+1
num_items = ml_100k['item_id'].nunique()+1

In [12]:
# construct the train and test datasets
data = NCF_Data(args, ml_100k)
train_loader = data.get_train_instance()
test_loader = data.get_test_instance()

# set model and loss, optimizer
model = NeuMF(args, num_users, num_items)
model = model.to(device)
loss_function = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)

# train, evaluation
best_hr = 0
for epoch in range(1, args.epochs+1):
    model.train() # Enable dropout (if have).
    start_time = time.time()

    for user, item, label in train_loader:
        user = user.to(device)
        item = item.to(device)
        label = label.to(device)

        optimizer.zero_grad()
        prediction = model(user, item)
        loss = loss_function(prediction, label)
        loss.backward()
        optimizer.step()
        #writer.add_scalar('loss/Train_loss', loss.item(), epoch)

    model.eval()
    HR, NDCG = metrics(model, test_loader, args.top_k, device)
    #writer.add_scalar('Perfomance/HR@10', HR, epoch)
    #writer.add_scalar('Perfomance/NDCG@10', NDCG, epoch)

    elapsed_time = time.time() - start_time
    print("The time elapse of epoch {:03d}".format(epoch) + " is: " + 
            time.strftime("%H: %M: %S", time.gmtime(elapsed_time)))
    print("HR: {:.3f}\tNDCG: {:.3f}".format(np.mean(HR), np.mean(NDCG)))

    if HR > best_hr:
        best_hr, best_ndcg, best_epoch = HR, NDCG, epoch
        if args.out:
            torch.save(model.state_dict(), "baseline.pt")
#writer.close()

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


The time elapse of epoch 001 is: 00: 00: 18
HR: 0.554	NDCG: 0.314
The time elapse of epoch 002 is: 00: 00: 18
HR: 0.585	NDCG: 0.333
The time elapse of epoch 003 is: 00: 00: 18
HR: 0.599	NDCG: 0.348
The time elapse of epoch 004 is: 00: 00: 19
HR: 0.625	NDCG: 0.362
The time elapse of epoch 005 is: 00: 00: 21
HR: 0.633	NDCG: 0.371
The time elapse of epoch 006 is: 00: 00: 18
HR: 0.619	NDCG: 0.363
The time elapse of epoch 007 is: 00: 00: 18
HR: 0.635	NDCG: 0.374
The time elapse of epoch 008 is: 00: 00: 18
HR: 0.637	NDCG: 0.370
The time elapse of epoch 009 is: 00: 00: 18
HR: 0.628	NDCG: 0.365
The time elapse of epoch 010 is: 00: 00: 18
HR: 0.622	NDCG: 0.362
The time elapse of epoch 011 is: 00: 00: 18
HR: 0.631	NDCG: 0.360
The time elapse of epoch 012 is: 00: 00: 18
HR: 0.629	NDCG: 0.363
The time elapse of epoch 013 is: 00: 00: 18
HR: 0.618	NDCG: 0.353
The time elapse of epoch 014 is: 00: 00: 18
HR: 0.617	NDCG: 0.354
The time elapse of epoch 015 is: 00: 00: 18
HR: 0.603	NDCG: 0.345
The time e

# Try predicting on our test set

In [23]:
# - get test data in Dataloader format
def get_test_dataloader(test_ratings_df):
        users, items, ratings = [], [], []

        for row in test_ratings_df.itertuples():
            users.append(int(row.user_id))
            items.append(int(row.item_id))
            ratings.append(float(row.rating))
            
        dataset = Rating_Datset(
            user_list=users,
            item_list=items,
            rating_list=ratings)
        return torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False, num_workers=4)

In [26]:
test_dataloader = get_test_dataloader(ml_100k_test)
# set the num_users, items
num_users_test = ml_100k_test['user_id'].nunique()+1
num_items_test = ml_100k_test['item_id'].nunique()+1
print("num_users_test", num_users_test, " num_items_test: ", num_items_test)

model = NeuMF(args, num_users, num_items)
model.load_state_dict(torch.load("baseline.pt"))
model.eval()
print("num_users: ", num_users, "num_items: ", num_items)

top_k = 10

for user, item, label in test_dataloader:
    user = user.to(device)
    item = item.to(device)
    print("user: ", len(user))
    print("item: ", len(item))

    predictions = model(user, item)
    #print("predictions len: ", predictions)
    #_, indices = torch.topk(predictions, top_k)
    #recommends = torch.take(item, indices).cpu().numpy().tolist()
    #print(user)
    #print(item)
    #print(recommends)
    

num_users_test 460  num_items_test:  1411
num_users:  944 num_items:  1651


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
item:  1
user:  1
i

KeyboardInterrupt: ignored

Average epoch time is 90 seconds on Nvidia T4 GPU. Both hit_rate and ndcg values improves initially for first 4 epochs and then converged to a local (or global, I hope) minima.

<!-- ------------------------ -->
## Congratulations
Duration: 2

Congratulations! You covered a lot of content and hopefully you have a better understanding of the working of neural matrix factorization model by now.

### What we've covered
- Create movielens dataset class in Pytorch
- Setting the evaluation criteria
- Architecture of neural matrix factorization model
- Train and evaluating a neural matrix factorization model

### Resources
- [Colab notebook](https://sparsh-ai.github.io/rec-tutorials/matrixfactorization%20movielens%20pytorch%20scratch/2021/04/21/rec-algo-ncf-pytorch-pyy0715.html)

### Next Steps (Learn More)
- Notebook based tutorials [here](https://sparsh-ai.github.io/rec-tutorials/)
- Read NMF Paper on [Arxiv](https://arxiv.org/abs/1511.06443)
- Continue learning by following [this](https://medium.com/@lz2576/a-first-look-at-recommendation-system-with-matrix-factorization-and-neural-nets-7e21e54295c) medium post

#### Have a Question?
- https://form.jotform.com/211377288388469

#### Github Issues
- https://github.com/sparsh-ai/reco-tutorials/issues